In [0]:
import numpy as np
import pandas as pd


In [94]:
# Reading Data
officeData = pd.read_csv('ra_data_classifier.csv',encoding= 'unicode_escape')
officeData.head()


,hid,chunk,has_space
0,33BFF6QPI1YEFYISIWWDVQKGH8RW3Z,"Landmark Center, 8th Fl",0
1,3HUR21WDDUCUK1K6HMLPN3U0ZBSYX0,Contact: The C3 team at MakemeC3@cic.us -- Add...,0
2,3566S7OX5D63FG3CNKAIFH62QX517B,"A powerful tool for developers, the MySQL Data...",0
3,3GV1I4SEO9CX1NTBXKN9TIFKTSQ6LN,"Easy access to T, Hubway, and parking",0
4,37SDSEDIN9P7FU8VXP2OTH2X1HB18Y,Check out our Private Offices,1


In [95]:
len(officeData)

100

In [96]:
officeData.shape

(100, 3)

In [0]:
officeData=officeData.drop(columns=['hid'])

In [98]:
officeData.groupby('has_space').count()

,chunk
has_space,
0,65
1,35


In [99]:
officeData_copy=officeData.copy()
print(officeData_copy)

                                                chunk  has_space
0                             Landmark Center, 8th Fl          0
1   Contact: The C3 team at MakemeC3@cic.us -- Add...          0
2   A powerful tool for developers, the MySQL Data...          0
3               Easy access to T, Hubway, and parking          0
4                       Check out our Private Offices          1
..                                                ...        ...
95              Cambridge Coworking CommunityåÊ("C3")          0
96  Vibrant neighborhood community ÛÒÛÒ restaura...          0
97  Rates: "Geek Premium", dedicated workspace for...          1
98  The re-shaping of the American workforce, init...          1
99  Usermin is webmail and much more! Usermin prov...          0

[100 rows x 2 columns]


In [100]:
officeData_copy['chunk'] = officeData_copy['chunk'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()

officeData_copy['chunk'] = officeData_copy['chunk'].str.lower()

officeData_copy['chunk'] = officeData_copy['chunk'].str.split()

officeData_copy['chunk'].head()

0                          [landmark, center, 8th, fl]
1    [contact, the, c3, team, at, makemec3, cic, us...
2    [a, powerful, tool, for, developers, the, mysq...
3          [easy, access, to, t, hubway, and, parking]
4                  [check, out, our, private, offices]
Name: chunk, dtype: object

In [101]:
officeData_copy['has_space'].value_counts() / officeData_copy.shape[0] * 100

0    65.0
1    35.0
Name: has_space, dtype: float64

# Spliting Test and Train Data

In [0]:
train_data = officeData_copy.sample(frac=0.7,random_state=1).reset_index(drop=True)
test_data = officeData_copy.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

In [103]:
train_data['has_space'].value_counts() / train_data.shape[0] * 100

0    64.285714
1    35.714286
Name: has_space, dtype: float64

In [104]:
train_data.shape

(70, 2)

In [105]:
test_data['has_space'].value_counts() / test_data.shape[0] * 100

0    70.0
1    30.0
Name: has_space, dtype: float64

In [106]:
test_data.shape

(30, 2)

# Creating list of all words

In [107]:
vocabulary = list(set(train_data['chunk'].sum()))
len(vocabulary)

1068

In [108]:
print(vocabulary)

['terminal', 'resident', '2600', 'directly', 'line', 'processing', 'ûò', 'special', 'categories', 'how', 'ûªve', 'åêright', 'design', 'locals', 'anne', 'views', 'its', 'let', 'printers', '5', 'changing', 'floor', 'valley', 'parking', 'month', 'harrison', '8th', 'cardamone', 'have', 'comes', 'creatives', 'before', 'menberships', 'applications', 'solutions', 'days', 'info', 'near', 'greater', 'lxc', 'databases', 'didn', 'b', 'sql', 'åêåê617', 'integrated', 'broadway', 'business', 'provides', 'together', 'firewalls', 'schedule', 'workspace', 'shuttle', 'data', 'o', 'change', '4', 'åêåêåêåêåêåêfree', 'mill', 'cleantech', '2', 'find', 'install', 'atari', 'day', 'exclaimed', 'boss', 'update', 'biography', 'makes', 'across', 'gathers', 'overlooking', 'england', 'loughlin', 'specific', 'unlimited', 'delegation', 's', 'around', 'seven', 'sign', 'established', '00am', 'majority', 'offer', 'visiting', '15', 'cove', 'flexible', 'said', 'abby', '7pm', 'workforce', 'start', 'files', 'network', 'chel

# Calculating frequency of words in each chunk

In [0]:
wordFrequency_per_chunk = pd.DataFrame([
    [row[0].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)

In [0]:
train_data = pd.concat([train_data.reset_index(), wordFrequency_per_chunk], axis=1).iloc[:,1:]

In [111]:
train_data.head()


,chunk,has_space,terminal,resident,2600,directly,line,processing,ûò,special,categories,how,ûªve,åêright,design,locals,anne,views,its,let,printers,5,changing,floor,valley,parking,month,harrison,8th,cardamone,have,comes,creatives,before,menberships,applications,solutions,days,info,near,...,rental,google,sf,external,devoted,worldwide,next,that,locations,300,environments,turnstone,suburban,which,move,everything,focal,hard,grandfather,firsthand,site,make,hill,part,buildings,sales,live,easy,initiated,25,march,skills,virtual,tech,copier,while,accelerated,casual,matures,driven
0,"[full, 350, month, 24, 7, access]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"[ûïwhen, we, first, started, people, could, cl...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"[article, by, evona, w, niewiadomska]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"[check, out, this, great, video, about, cowork...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"[if, you, would, like, to, host, an, event, in...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [112]:
#co-efficient of cases where the word does not occur in vocabulary
alpha = 1

#total number of words in the dataset
vocabulary_size = len(vocabulary) 

#part of chunk in dataset which are labeled as renting space
renter = train_data['has_space'].value_counts()[1] / train_data.shape[0]

#part of chunk in dataset which are labeled as not renting space
non_renter = train_data['has_space'].value_counts()[0] / train_data.shape[0]

#Total number of words in chunks which are labeled as renting a space
Renter_wordCount = train_data.loc[train_data['has_space'] == 1, 'chunk'].apply(len).sum()

#Total number of words in chunks which are labeled as not renting a space
NonRent_WordCount = train_data.loc[train_data['has_space'] == 0, 'chunk'].apply(len).sum()
vocabulary_size

1068

In [0]:
def probability_word_rentedChunk(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['has_space'] == 1, word].sum() + alpha) / (Renter_wordCount + alpha*vocabulary_size)
    else:
        return 1

In [0]:
def probability_word_nonRentedChunk(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['has_space'] == 0, word].sum() + alpha) / (NonRent_WordCount + alpha*vocabulary_size)
    else:
        return 1

# Creating classifier and predicting on test data

In [0]:
def classify(message):
    probability_renter_given_message = renter 

    probability_nonrenter_given_message = non_renter
    for word in message:
        probability_renter_given_message *= probability_word_rentedChunk(word)
        probability_nonrenter_given_message *= probability_word_nonRentedChunk(word)
    if probability_nonrenter_given_message > probability_renter_given_message:
        return 0
    elif probability_nonrenter_given_message < probability_renter_given_message:
        return 1
    else:
        return 1

In [116]:
classify("Office Space available for rent")

1

In [117]:
test_data['predicted'] = test_data['chunk'].apply(classify)

test_data

,chunk,has_space,predicted
0,"[details, åêrent, includes, desk, and, chair, ...",1,1
1,"[lili, has, a, rich, history, in, fortune, tel...",0,0
2,"[configure, monitoring, of, systems, and, serv...",0,0
3,"[ûïthe, majority, of, our, host, locations, ar...",1,1
4,"[your, team, deserves, sunlight, nearly, all, ...",1,1
5,"[workshop, brookline, ûò, 400, month, full, ti...",1,1
6,"[not, required, åêåêno, notice, to, leave, jus...",0,0
7,"[the, new, default, theme, presents, systems, ...",0,0
8,"[special, pricing, on, multi, month, menbershi...",0,0
9,"[january, 1st, marked, the, opening, for, appl...",0,1


In [0]:
prediction_accuracy = (test_data['predicted'] == test_data['has_space']).sum() / test_data.shape[0] * 100

In [119]:
test_data.loc[test_data['predicted'] != test_data['has_space']]

,chunk,has_space,predicted
9,"[january, 1st, marked, the, opening, for, appl...",0,1
26,"[vibrant, neighborhood, community, ûò, ûò, res...",0,1


In [120]:
print(prediction_accuracy)

93.33333333333333
